In [1]:
import pandas as pd

In [2]:
lots = pd.read_csv('data/Lots & Permissions CH3.csv')

In [3]:
# Drop rows from index 236 onwards
lots = lots.drop(lots.index[236:])

In [ ]:
# Disable pandas truncation to see all columns
pd.set_option('display.max_columns', None)

# Print all permission columns
print("All permission columns:")
permission_columns = lots.columns[8:]
for i, col in enumerate(permission_columns, 1):
    print(f"{i:3d}. {col}")

# Print total count
print(f"\nTotal number of permission columns: {len(permission_columns)}")

# Optional: Reset display settings
# pd.reset_option('display.max_columns')

In [ ]:
lots[lots['No Permits'] == 1]

In [ ]:
# First, identify all permission columns (columns after and including '17FAE')
permission_columns = lots.columns[lots.columns.get_loc('17FAE'):]

# Create a copy of the DataFrame
lots_modified = lots.copy()

# Where 'Everyone' is 1, set all permission columns to 1
mask = lots_modified['Everyone'] == 1.0
for col in permission_columns:
    lots_modified.loc[mask, col] = 1.0

# Update Count Valid Permissions for affected rows
lots_modified.loc[mask, 'Count Valid Permissions in Lot by Date/Time'] = len(permission_columns)

# Drop the 'Everyone' column
lots_modified = lots_modified.drop('Everyone', axis=1)

# Verify the changes
print(f"Original DataFrame shape: {lots.shape}")
print(f"Modified DataFrame shape: {lots_modified.shape}")
print(f"\nNumber of rows where 'Everyone' was 1: {mask.sum()}")
print(f"Number of permission columns set to 1: {len(permission_columns)}")

# Optional: Show a sample row where changes were made
if mask.any():
    print("\nSample of modified row:")
    sample_row = lots_modified[mask].iloc[0]
    print(f"Parking Lot: {sample_row['Parking Lot / Zone Name']}")
    print(f"Time: {sample_row['Start Time - Daily']} - {sample_row['End Time - Daily']}")
    print(f"Count Valid Permissions: {sample_row['Count Valid Permissions in Lot by Date/Time']}")
    print("First few permission values:", sample_row[permission_columns[:5]].to_dict())

lots = lots_modified

In [ ]:
# First, identify all permission columns (columns after and including '17FAE')
permission_columns = lots.columns[lots.columns.get_loc('17FAE'):]

# Create a copy of the DataFrame
lots_modified = lots.copy()

# Where 'All Permits' is 1, set all permission columns to 1 (except 'No Permits')
mask = lots_modified['All Permits'] == 1.0
for col in permission_columns:
    if col != 'No Permits':  # Skip 'No Permits' column
        lots_modified.loc[mask, col] = 1.0

# Update Count Valid Permissions for affected rows
# Subtract 1 from len(permission_columns) to account for 'No Permits'
lots_modified.loc[mask, 'Count Valid Permissions in Lot by Date/Time'] = len(permission_columns) - 1

# Drop the 'All Permits' column
lots_modified = lots_modified.drop('All Permits', axis=1)

# Verify the changes
print(f"Original DataFrame shape: {lots.shape}")
print(f"Modified DataFrame shape: {lots_modified.shape}")
print(f"\nNumber of rows where 'All Permits' was 1: {mask.sum()}")
print(f"Number of permission columns set to 1: {len(permission_columns) - 1}")  # minus 'No Permits'

# Optional: Show a sample row where changes were made
if mask.any():
    print("\nSample of modified row:")
    sample_row = lots_modified[mask].iloc[0]
    print(f"Parking Lot: {sample_row['Parking Lot / Zone Name']}")
    print(f"Time: {sample_row['Start Time - Daily']} - {sample_row['End Time - Daily']}")
    print(f"Count Valid Permissions: {sample_row['Count Valid Permissions in Lot by Date/Time']}")
    print("First few permission values:", sample_row[permission_columns[:5]].to_dict())

lots = lots_modified

In [ ]:
lots.drop('Unnamed: 0', axis=1, inplace=True)

In [10]:
lots.rename(columns={'Parking Lot / Zone Name': 'Lot Name'}, inplace=True)

In [11]:
lots.to_csv('data/Lots & Permissions CH3.csv', index=False)

In [ ]:
lots[(lots['Lot Name'] == 'Y') & (lots['Enforcement Days'] == 'Weekdays')][['Start Time - Daily', 'End Time - Daily']]

In [ ]:
# First, make sure we're using the correct column name
# It might be 'Parking Lot / Zone Name' instead of 'Lot Name'
lot_name_col = 'Parking Lot / Zone Name' if 'Parking Lot / Zone Name' in lots.columns else 'Lot Name'

# Get unique lot names
unique_lots = lots[lots['Enforcement Days'] == 'Weekdays'][lot_name_col].unique()

# Function to check for time overlaps
def has_time_overlap(times):
    """Check if there are overlapping time periods in the list of (start, end) times."""
    for i, (start1, end1) in enumerate(times):
        for j, (start2, end2) in enumerate(times):
            if i != j:  # Don't compare a time period with itself
                # Convert times to 24-hour format for comparison
                start1_hours = int(start1.split(':')[0]) % 24
                end1_hours = int(end1.split(':')[0]) % 24
                start2_hours = int(start2.split(':')[0]) % 24
                end2_hours = int(end2.split(':')[0]) % 24
                
                # Handle special case where end time is 00:00:00 (midnight)
                if end1_hours == 0:
                    end1_hours = 24
                if end2_hours == 0:
                    end2_hours = 24
                
                # Check for overlap
                # If period 1 starts during period 2, or period 2 starts during period 1
                if ((start1_hours < end2_hours and start1_hours >= start2_hours) or
                    (start2_hours < end1_hours and start2_hours >= start1_hours)):
                    return True
    return False

# Find lots with time overlaps
lots_with_overlaps = []

for lot in unique_lots:
    lot_data = lots[(lots[lot_name_col] == lot) & (lots['Enforcement Days'] == 'Weekdays')]
    
    # Skip lots with only one time period
    if len(lot_data) <= 1:
        continue
    
    # Get time periods for this lot
    time_periods = list(zip(lot_data['Start Time - Daily'], lot_data['End Time - Daily']))
    
    # Check for overlaps
    if has_time_overlap(time_periods):
        lots_with_overlaps.append({
            'Lot Name': lot,
            'Time Periods': time_periods,
            'Count': len(time_periods)
        })

# Print results
print(f"Found {len(lots_with_overlaps)} lots with time overlaps on Weekdays:")
for i, lot_info in enumerate(lots_with_overlaps, 1):
    print(f"\n{i}. Lot: {lot_info['Lot Name']} ({lot_info['Count']} time periods)")
    print("   Time periods:")
    for start, end in lot_info['Time Periods']:
        print(f"   - {start} to {end}")

# Show detailed data for all problematic lots
if lots_with_overlaps:
    print("\nDetailed data for all problematic lots:")
    for lot_info in lots_with_overlaps:
        lot_name = lot_info['Lot Name']
        print(f"\nLot: {lot_name}")
        print(lots[(lots[lot_name_col] == lot_name) & (lots['Enforcement Days'] == 'Weekdays')][
            [lot_name_col, 'Start Time - Daily', 'End Time - Daily', 'Posted Restrictions']
        ])

___

In [1]:
import pandas as pd
lots = pd.read_csv('data/Lots & Permissions CH3.csv')

In [2]:
lots

,Lot Type,Physical Location (Yes/No),Lot Name,Posted Restrictions,Enforcement Days,Start Time - Daily,End Time - Daily,Count Valid Permissions in Lot by Date/Time,17FAE,18-19FAE,...,Visitor After Hours,VM,W,WW,X,XX,Y,YC,YY,Z
0,Permitted,YES,A,Restricted after 4PM,Weekends,00:00:00,00:00:00,59,0,1,...,0,1,1,1,1,1,1,1,1,1
1,Permitted,YES,A,Restricted after 4PM,Weekdays,07:00:00,16:00:00,8,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Permitted,YES,A,Restricted after 4PM,Weekdays,16:00:00,07:00:00,60,0,1,...,0,1,1,1,1,1,1,1,1,1
3,Special Restrictions,YES,AA,Special Restrictions,Weekends,00:00:00,00:00:00,62,0,1,...,0,1,1,1,1,1,1,1,1,1
4,Special Restrictions,YES,AA,Special Restrictions,Weekdays,07:00:00,16:00:00,7,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,Permitted,YES,Y,Modified restricted,Weekdays,16:00:00,07:00:00,1,0,0,...,0,0,0,0,0,0,1,0,0,0
232,Permitted,YES,Y,Modified restricted,Weekdays,16:00:00,20:00:00,60,0,1,...,0,1,1,1,1,1,1,1,1,1
233,Special Restrictions,YES,YC,Special Restrictions,Weekdays,00:00:00,18:00:00,7,0,1,...,0,0,0,0,0,0,0,1,0,0
234,Permitted,YES,YY,Unrestricted after 4PM,Always,00:00:00,00:00:00,7,0,1,...,0,0,0,0,0,0,0,0,1,0


In [3]:
# drop rows where Lot Name is in ['Lot MM', 'S7/8', 'R', 'N1/4/6/8/9', 'LL', 'TT', 'SS'] and Enforcement Days is not 'Always'
lots.drop(lots.index[[43, 44, 156, 157, 183, 184, 191, 192, 200, 201, 209, 210]], inplace=True)

In [4]:
lots.reset_index(drop=True, inplace=True)
lots[lots['Lot Name'] == 'Y']

,Lot Type,Physical Location (Yes/No),Lot Name,Posted Restrictions,Enforcement Days,Start Time - Daily,End Time - Daily,Count Valid Permissions in Lot by Date/Time,17FAE,18-19FAE,...,Visitor After Hours,VM,W,WW,X,XX,Y,YC,YY,Z
217,Permitted,YES,Y,Modified restricted,Weekends,00:00:00,00:00:00,61,0,1,...,0,1,1,1,1,1,1,1,1,1
218,Permitted,YES,Y,Modified restricted,Weekdays,07:00:00,16:00:00,7,0,1,...,0,0,0,0,0,0,1,0,0,0
219,Permitted,YES,Y,Modified restricted,Weekdays,16:00:00,07:00:00,1,0,0,...,0,0,0,0,0,0,1,0,0,0
220,Permitted,YES,Y,Modified restricted,Weekdays,16:00:00,20:00:00,60,0,1,...,0,1,1,1,1,1,1,1,1,1


In [5]:
lots[lots['Lot Name'] == 'II']

,Lot Type,Physical Location (Yes/No),Lot Name,Posted Restrictions,Enforcement Days,Start Time - Daily,End Time - Daily,Count Valid Permissions in Lot by Date/Time,17FAE,18-19FAE,...,Visitor After Hours,VM,W,WW,X,XX,Y,YC,YY,Z
34,Permitted,YES,II,24-Hour restricted,Weekends,00:00:00,00:00:00,61,0,1,...,0,1,1,1,1,1,1,1,1,1
35,Permitted,YES,II,24-Hour restricted,Weekdays,07:00:00,16:00:00,7,0,1,...,0,0,0,0,0,0,0,0,0,0
36,Permitted,YES,II,24-Hour restricted,Weekdays,16:00:00,20:00:00,61,0,1,...,0,1,1,1,1,1,1,1,1,1


In [6]:
# For rows 34, 35, and 36, make all columns after the 8th column 0, except for 'No Permits'
row_indices = [34, 35, 36]
cols_to_keep = ['No Permits']
cols_to_modify = lots.columns[8:]

for row_index in row_indices:
    for col in cols_to_modify:
        if col not in cols_to_keep:
            lots.at[row_index, col] = 0

    # Update the value of 'No Permits' to 1
    lots.at[row_index, 'No Permits'] = 1

    # Update the sum of 'Count Valid Permissions in Lot by Date/Time'
    lots.at[row_index, 'Count Valid Permissions in Lot by Date/Time'] = lots.loc[row_index, cols_to_modify].sum()


In [7]:
lots[lots['Lot Name'] == 'II']

,Lot Type,Physical Location (Yes/No),Lot Name,Posted Restrictions,Enforcement Days,Start Time - Daily,End Time - Daily,Count Valid Permissions in Lot by Date/Time,17FAE,18-19FAE,...,Visitor After Hours,VM,W,WW,X,XX,Y,YC,YY,Z
34,Permitted,YES,II,24-Hour restricted,Weekends,00:00:00,00:00:00,1,0,0,...,0,0,0,0,0,0,0,0,0,0
35,Permitted,YES,II,24-Hour restricted,Weekdays,07:00:00,16:00:00,1,0,0,...,0,0,0,0,0,0,0,0,0,0
36,Permitted,YES,II,24-Hour restricted,Weekdays,16:00:00,20:00:00,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
lots[lots['Lot Name'] == 'Q1']

,Lot Type,Physical Location (Yes/No),Lot Name,Posted Restrictions,Enforcement Days,Start Time - Daily,End Time - Daily,Count Valid Permissions in Lot by Date/Time,17FAE,18-19FAE,...,Visitor After Hours,VM,W,WW,X,XX,Y,YC,YY,Z
172,Permitted,YES,Q1,Modified restricted,Weekends,00:00:00,00:00:00,55,0,1,...,0,0,1,1,1,1,1,1,1,1
173,Permitted,YES,Q1,Modified restricted,Weekdays,07:00:00,16:00:00,6,0,1,...,0,0,0,0,0,0,0,0,0,0
174,Permitted,YES,Q1,Modified restricted,Weekdays,16:00:00,20:00:00,55,0,1,...,0,0,1,1,1,1,1,1,1,1


In [9]:
row_indices = lots.index[lots['Lot Name'] == 'Q1']
cols_to_keep = ['Q']
cols_to_modify = lots.columns[8:]

for row_index in row_indices:
    for col in cols_to_modify:
        if col not in cols_to_keep:
            lots.at[row_index, col] = 0

    # Ensure 'Q' is set to 1
    lots.at[row_index, 'Q'] = 1

    # Update the sum of 'Count Valid Permissions in Lot by Date/Time'
    lots.at[row_index, 'Count Valid Permissions in Lot by Date/Time'] = lots.loc[row_index, cols_to_modify].sum()

In [10]:
lots[lots['Lot Name'] == 'Q1']

,Lot Type,Physical Location (Yes/No),Lot Name,Posted Restrictions,Enforcement Days,Start Time - Daily,End Time - Daily,Count Valid Permissions in Lot by Date/Time,17FAE,18-19FAE,...,Visitor After Hours,VM,W,WW,X,XX,Y,YC,YY,Z
172,Permitted,YES,Q1,Modified restricted,Weekends,00:00:00,00:00:00,1,0,0,...,0,0,0,0,0,0,0,0,0,0
173,Permitted,YES,Q1,Modified restricted,Weekdays,07:00:00,16:00:00,1,0,0,...,0,0,0,0,0,0,0,0,0,0
174,Permitted,YES,Q1,Modified restricted,Weekdays,16:00:00,20:00:00,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
lots[lots['Lot Name'] == 'Q']

,Lot Type,Physical Location (Yes/No),Lot Name,Posted Restrictions,Enforcement Days,Start Time - Daily,End Time - Daily,Count Valid Permissions in Lot by Date/Time,17FAE,18-19FAE,...,Visitor After Hours,VM,W,WW,X,XX,Y,YC,YY,Z
169,Permitted,YES,Q,Unrestricted after 4PM,Weekends,00:00:00,00:00:00,61,0,1,...,0,1,1,1,1,1,1,1,1,1
170,Permitted,YES,Q,Unrestricted after 4PM,Weekdays,07:00:00,16:00:00,7,0,1,...,0,0,0,0,0,0,0,0,0,0
171,Permitted,YES,Q,Unrestricted after 4PM,Weekdays,16:00:00,20:00:00,61,0,1,...,0,1,1,1,1,1,1,1,1,1


In [12]:
lots[lots['Lot Name'] == 'YC']

,Lot Type,Physical Location (Yes/No),Lot Name,Posted Restrictions,Enforcement Days,Start Time - Daily,End Time - Daily,Count Valid Permissions in Lot by Date/Time,17FAE,18-19FAE,...,Visitor After Hours,VM,W,WW,X,XX,Y,YC,YY,Z
221,Special Restrictions,YES,YC,Special Restrictions,Weekdays,00:00:00,18:00:00,7,0,1,...,0,0,0,0,0,0,0,1,0,0


In [13]:
# Find the row with 'Lot Name' == 'YC'
yc_row = lots[lots['Lot Name'] == 'YC'].copy()

# Modify the necessary columns
yc_row['Start Time - Daily'] = '18:00:00'
yc_row['End Time - Daily'] = '00:00:00'
yc_row['Posted Restrictions'] = 'Unrestricted after 4PM'

# Set all permit columns to 1 except for 'Everyone'
permit_columns = yc_row.columns[8:]
for col in permit_columns:
    yc_row[col] = 1

# Update the sum of 'Count Valid Permissions in Lot by Date/Time'
yc_row['Count Valid Permissions in Lot by Date/Time'] = yc_row[permit_columns].sum(axis=1)

# Append the modified row to the dataframe
lots = pd.concat([lots, yc_row], ignore_index=True)


In [14]:
lots[lots['Lot Name'] == 'YC']

,Lot Type,Physical Location (Yes/No),Lot Name,Posted Restrictions,Enforcement Days,Start Time - Daily,End Time - Daily,Count Valid Permissions in Lot by Date/Time,17FAE,18-19FAE,...,Visitor After Hours,VM,W,WW,X,XX,Y,YC,YY,Z
221,Special Restrictions,YES,YC,Special Restrictions,Weekdays,00:00:00,18:00:00,7,0,1,...,0,0,0,0,0,0,0,1,0,0
224,Special Restrictions,YES,YC,Unrestricted after 4PM,Weekdays,18:00:00,00:00:00,126,1,1,...,1,1,1,1,1,1,1,1,1,1


In [15]:
lots.to_csv('data/Lots & Permissions CH4.csv', index=False)

---

In [1]:
import pandas as pd
lots = pd.read_csv('data/Lots_Permissions_CH5_fakedata (1).csv')
lots.head()

,Lot Type,Physical Location (Yes/No),Lot Name,Posted Restrictions,Enforcement Days,Start Time - Daily,End Time - Daily,Count Valid Permissions in Lot by Date/Time,17FAE,18-19FAE,...,Visitor After Hours,VM,W,WW,X,XX,Y,YC,YY,Z
0,Permitted,YES,A,Restricted after 4PM,Weekends,0:00:00,0:00:00,58,0,1,...,0,1,1,1,1,1,1,1,1,1
1,Permitted,YES,A,Restricted after 4PM,Weekdays,7:00:00,16:00:00,8,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Permitted,YES,A,Restricted after 4PM,Weekdays,16:00:00,7:00:00,59,0,1,...,0,1,1,1,1,1,1,1,1,1
3,Special Restrictions,YES,AA,Special Restrictions,Weekends,0:00:00,0:00:00,61,0,1,...,0,1,1,1,1,1,1,1,1,1
4,Special Restrictions,YES,AA,Special Restrictions,Weekdays,7:00:00,16:00:00,7,0,1,...,0,0,0,0,0,0,0,0,0,0


In [31]:
lots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Columns: 134 entries, Lot Type  to Z
dtypes: int64(127), object(7)
memory usage: 282.8+ KB


In [32]:
x = lots.iloc[:, :7]

In [33]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Lot Type                    270 non-null    object
 1   Physical Location (Yes/No)  270 non-null    object
 2   Lot Name                    270 non-null    object
 3   Posted Restrictions         269 non-null    object
 4   Enforcement Days            270 non-null    object
 5   Start Time - Daily          270 non-null    object
 6   End Time - Daily            270 non-null    object
dtypes: object(7)
memory usage: 14.9+ KB


In [39]:
# lots.rename(columns={'Lot Type': 'Lot Type '}, inplace=True)

In [40]:
# lots.to_csv('data/Lots_Permissions_CH5_fakedata (1).csv', index=False)